In [ ]:
"""
Purpose: To decompose the multi-somas for splitting
using the new decomposition method



"""


In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

INFO - 2021-03-19 17:22:31,018 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-19 17:22:31,020 - settings - Setting database.user to celiib
INFO - 2021-03-19 17:22:31,021 - settings - Setting database.password to newceliipass
INFO - 2021-03-19 17:22:31,025 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-03-19 17:22:31,025 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-19 17:22:31,038 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-03-19 17:22:31,439 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-19 17:22:31,443 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-19 17:22:31,443 - settings - Setting database.user to celiib
INFO - 2021-03-19 17:22:31,444 - settings - Setting database.password to newceliipass
INFO - 2021-03-19 17:22:31,444 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-19 17:22:31,445 - settings - Setting database.user to celiib
INFO - 2021-03-19 17:22:31,445 - settings - Setting database.password to newceliipass
INFO - 2021-03-19 17:22:31,447 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-19 17:22:31,861 - settings - Setting enable_python_native_blobs to True


In [2]:
#so that it will have the adapter defined
from datajoint_utils import *

In [3]:
test_mode = False

# Debugging the contains method

In [4]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-03-19 17:22:31,959 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-19 17:22:31,960 - settings - Setting database.user to celiib
INFO - 2021-03-19 17:22:31,962 - settings - Setting database.password to newceliipass
INFO - 2021-03-19 17:22:31,963 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-19 17:22:31,964 - settings - Setting database.user to celiib
INFO - 2021-03-19 17:22:31,965 - settings - Setting database.password to newceliipass
INFO - 2021-03-19 17:22:31,967 - settings - Setting enable_python_native_blobs to True


Sleeping 83 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-03-19 17:22:32,417 - settings - Setting enable_python_native_blobs to True


In [5]:
#(schema.jobs & "table_name='__decomposition'").delete()

# Defining the Table

In [6]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [7]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 9299


In [8]:
decimation_version = 0
decimation_ratio = 0.25

current_allen_table = (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)).proj()
key_source = ((minnie.Decimation).proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0")  & 
              current_allen_table)
key_source                 

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses)
864691134988608634,0,0.25
864691135065024068,0,0.25
864691135155711844,0,0.25
864691135210232512,0,0.25
864691135341057989,0,0.25
864691135345291423,0,0.25
864691135373423816,0,0.25
864691135373676873,0,0.25
864691135382990571,0,0.25
864691135463533125,0,0.25


In [9]:
#minnie.DecompositionVersions.insert1(dict(process_version=6,description="3/4: retry limb processing with meshparty if meshafterparty fails"),skip_duplicates=True)

In [10]:
# schema.external['decomposition'].delete(delete_external_files=True)
# schema.external['somas'].delete(delete_external_files=True)
# schema.external['faces'].delete(delete_external_files=True)
# # schema.external['decimated_meshes'].delete(delete_external_files=True)

In [13]:
# key_source = ((minnie.Decimation).proj(decimation_version='version') & 
#                             "decimation_version=" + str(decimation_version) &
#                        f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0")  & (minnie.AllenProofreading() & dict(month=3,day=18,year=2021)).proj())               
# minnie.Decomposition() & key_source

In [14]:
import numpy as np
import time
decimation_version = 0
decimation_ratio = 0.25
process_version = 6

@schema
class Decomposition(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    ver : decimal(6,2) #the version number of the materializaiton
    process_version : int unsigned #the version of the preprocessing pipeline run
    index : tinyint unsigned  #the index of the neuron object that resulted from that mesh (indexed starting at 0)
    ---
    multiplicity=null    : tinyint unsigned             # the number of somas found for this base segment
    decomposition: <decomposition>
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_not_processed_soma_containing_meshes : int unsigned  #the number of meshes with somas that were not processed
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branches
    
    run_time=NULL : double                   # the amount of time to run (seconds)

    
    """


    
    key_source = ((minnie.Decimation).proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0")  & (minnie.AllenProofreading() & dict(month=3,day=18,year=2021)).proj())               
    

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decimated mesh
        3) Get the somas info
        4) Run the preprocessing
        5) Calculate all starter stats
        6) Save the file in a certain location
        7) Pass stats and file location to insert
        """
        whole_pass_time = time.time()
        #1) Get the segment id from the key
        segment_id = key["segment_id"]
        description = str(key['decimation_version']) + "_25"
        print(f"\n\n\n---- Working on Neuron {key['segment_id']} ----")
        global_start = time.time()
        
        #2) Get the decimated mesh
        current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)
        

        #3) Get the somas info 
        somas = du.get_soma_mesh_list(segment_id) 
        soma_ver = du.get_soma_mesh_list_ver(segment_id)
                  
        
        print(f"somas = {somas}")
                  
        #3b) Get the glia and nuclei information 
        glia_faces,nuclei_faces = du.get_segment_glia_nuclei_faces(segment_id,return_empty_list=True)
        
                  
        #4) Run the preprocessing


        total_neuron_process_time = time.time()

        print(f"\n--- Beginning preprocessing of {segment_id}---")
        recovered_neuron = neuron.Neuron(
        mesh = current_neuron_mesh,
        somas = somas,
        segment_id=segment_id,
        description=description,
        suppress_preprocessing_print=False,
        suppress_output=False,
        calculate_spines=True,
        widths_to_calculate=["no_spine_median_mesh_center"],
        glia_faces=glia_faces,
        nuclei_faces = nuclei_faces,

                )

        print(f"\n\n\n---- Total preprocessing time = {time.time() - total_neuron_process_time}")


        #5) Don't have to do any of the processing anymore because will do in the neuron object
        stats_dict = recovered_neuron.neuron_stats()



        #6) Save the file in a certain location
        save_time = time.time()
        ret_file_path = recovered_neuron.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                          return_file_path=True,
                                         export_mesh=False,
                                         suppress_output=True)

        ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
        print(f"Save time = {time.time() - save_time}")

        #7) Pass stats and file location to insert
        new_key = dict(key,
                       ver = soma_ver,
                       process_version = process_version,
                       index = 0,
                       multiplicity=1,
                       decomposition=ret_file_path_str,
                       n_vertices=len(current_neuron_mesh.vertices),
                       n_faces=len(current_neuron_mesh.faces),
                       run_time=np.round(time.time() - whole_pass_time,4)
                      )
        new_key.update(stats_dict)
        
        keys_to_delete = ["axon_length",
        "axon_area",
        "max_soma_volume",
        "max_soma_n_faces"]
        
        for k_to_delete in keys_to_delete:
            del new_key[k_to_delete]

        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)

        print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")
    

# Running the Populate

In [20]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition'")
(curr_table)#.delete()
#curr_table.delete()
#(curr_table & "error_message = 'ValueError: need at least one array to concatenate'").delete()

In [18]:
import time
import random
import compartment_utils as cu
cu = reload(cu)
import preprocessing_vp2 as pre
pre = reload(pre)
import skeleton_utils as sk
sk = reload(sk)
nru = reload(nru)

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    Decomposition.populate(reserve_jobs=True, suppress_errors=True)
else:
    Decomposition.populate(reserve_jobs=True, suppress_errors=False)
print('Populate Done')

print(f"Total time for DecompositionMultiSoma populate = {time.time() - start_time}")

INFO - 2021-03-19 17:23:45,251 - autopopulate - Found 33 keys to populate
INFO - 2021-03-19 17:23:45,263 - connection - Transaction started
INFO - 2021-03-19 17:23:45,265 - autopopulate - Populating: {'segment_id': 864691135081756919, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25')}


Populate Started



---- Working on Neuron 864691135081756919 ----
somas = [[<trimesh.Trimesh(vertices.shape=(36370, 3), faces.shape=(70209, 3))>], array([1519.097]), array([0.642])]

--- Beginning preprocessing of 864691135081756919---
--- 0) Having to preprocess the Neuron becuase no preprocessed data
Please wait this could take a while.....
Skipping the hole filling
use_meshafterparty = True
Using pre-computed somas: soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(36370, 3), faces.shape=(70209, 3))>]
Soma List = [<trimesh.Trimesh(vertices.shape=(36370, 3), faces.shape=(70209, 3))>]
soma_mesh_list_centers = [array([819939.16994225, 737385.68776739, 717178.46255705])]
Getting Glia and Nuclei Pieces Subtracted Away 3.4213850498199463
 Splitting mesh after soma cancellation 7.530466556549072
# of split_meshes = 33
 Containing Mesh Indices 0.1404116153717041
 non_soma_touching_meshes 2.7179718017578125e-05
There were 32 pieces found after size threshold
 Finding inside pieces and non_

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/542447 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:42<00:00, 42.42s/it]


...

  0%|          | 0/72 [00:00<?, ?it/s]

Checking connected components after clean_skeleton
after DISTANCE cleaning limb size of skeleton = (3339, 2, 3)
********COMBINING CLOSE SKELETON NODES WITHIN 700 DISTANCE**********
combine_threshold = 700
Found 1 valid paths to replace
valid_paths = [array([773., 781.])]
valid_path_lengths = [229.49291928074817]
length of Graph = 3340
Working on path [773. 781.]
path_degrees = [3, 3]
skeletonize_and_clean_connected_branch_CGAL: 75.66171908378601
Working on limb correspondence for #0 MAP piece
xvfb-run -n 9299 -s "-screen 0 800x600x24" meshlabserver $@  -i /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/temp/neuron_25006.off -o /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/temp/neuron_25006_remove_interior.off -s /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/temp/remove_interior_26814.mls
removed temporary input file: /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipel

0it [00:00, ?it/s]

empty mesh_correspondence_indices_2 returned so returning original mesh correspondence
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-18-b471a184886a>", line 18, in <module>
    Decomposition.populate(reserve_jobs=True, suppress_errors=False)
  File "/usr/local/lib/python3.6/dist-packages/datajoint/autopopulate.py", line 159, in populate
    make(dict(key))
  File "<ipython-input-14-d3011e153ab9>", line 118, in make
    nuclei_faces = nuclei_faces,
  File "/meshAfterParty/meshAfterParty/neuron.py", line 2001, in __init__
    nuclei_faces = nuclei_faces)
  File "/meshAfterParty/meshAfterParty/preprocessing_vp2.py", line 3712, in preprocess_neuron
    error_on_no_starting_coordinates=True
  File "/meshAfterParty/meshAfterParty/preprocessing_vp2.py", line 1293, in preprocess_limb
    remove_inside_pieces_threshold=100)
  

TypeError: object of type 'NoneType' has no len()